## Context:

Access to safe drinking-water is essential to health, a basic human right and a component of effective policy for health protection. This is important as a health and development issue at a national, regional and local level. In some regions, it has been shown that investments in water supply and sanitation can yield a net economic benefit, since the reductions in adverse health effects and health care costs outweigh the costs of undertaking the interventions.

Content
The water_potability.csv file contains water quality metrics for 3276 different water bodies.

1. pH value:
PH is an important parameter in evaluating the acid–base balance of water. It is also the indicator of acidic or alkaline condition of water status. WHO has recommended maximum permissible limit of pH from 6.5 to 8.5. The current investigation ranges were 6.52–6.83 which are in the range of WHO standards.

2. Hardness:
Hardness is mainly caused by calcium and magnesium salts. These salts are dissolved from geologic deposits through which water travels. The length of time water is in contact with hardness producing material helps determine how much hardness there is in raw water. Hardness was originally defined as the capacity of water to precipitate soap caused by Calcium and Magnesium.

3. Solids (Total dissolved solids - TDS):
Water has the ability to dissolve a wide range of inorganic and some organic minerals or salts such as potassium, calcium, sodium, bicarbonates, chlorides, magnesium, sulfates etc. These minerals produced un-wanted taste and diluted color in appearance of water. This is the important parameter for the use of water. The water with high TDS value indicates that water is highly mineralized. Desirable limit for TDS is 500 mg/l and maximum limit is 1000 mg/l which prescribed for drinking purpose.

4. Chloramines:
Chlorine and chloramine are the major disinfectants used in public water systems. Chloramines are most commonly formed when ammonia is added to chlorine to treat drinking water. Chlorine levels up to 4 milligrams per liter (mg/L or 4 parts per million (ppm)) are considered safe in drinking water.

5. Sulfate:
Sulfates are naturally occurring substances that are found in minerals, soil, and rocks. They are present in ambient air, groundwater, plants, and food. The principal commercial use of sulfate is in the chemical industry. Sulfate concentration in seawater is about 2,700 milligrams per liter (mg/L). It ranges from 3 to 30 mg/L in most freshwater supplies, although much higher concentrations (1000 mg/L) are found in some geographic locations.

6. Conductivity:
Pure water is not a good conductor of electric current rather’s a good insulator. Increase in ions concentration enhances the electrical conductivity of water. Generally, the amount of dissolved solids in water determines the electrical conductivity. Electrical conductivity (EC) actually measures the ionic process of a solution that enables it to transmit current. According to WHO standards, EC value should not exceeded 400 μS/cm.

7. Organic_carbon:
Total Organic Carbon (TOC) in source waters comes from decaying natural organic matter (NOM) as well as synthetic sources. TOC is a measure of the total amount of carbon in organic compounds in pure water. According to US EPA < 2 mg/L as TOC in treated / drinking water, and < 4 mg/Lit in source water which is use for treatment.

8. Trihalomethanes:
THMs are chemicals which may be found in water treated with chlorine. The concentration of THMs in drinking water varies according to the level of organic material in the water, the amount of chlorine required to treat the water, and the temperature of the water that is being treated. THM levels up to 80 ppm is considered safe in drinking water.

9. Turbidity:
The turbidity of water depends on the quantity of solid matter present in the suspended state. It is a measure of light emitting properties of water and the test is used to indicate the quality of waste discharge with respect to colloidal matter. The mean turbidity value obtained for Wondo Genet Campus (0.98 NTU) is lower than the WHO recommended value of 5.00 NTU.

10. Potability:
Indicates if water is safe for human consumption where 1 means Potable and 0 means Not potable.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib  inline
sns.set(color_codes=True)

In [ ]:
plt.rcParams['axes.labelsize'] = 15.
plt.rcParams['xtick.labelsize'] = 15.
plt.rcParams['ytick.labelsize']= 15.
plt.rcParams['legend.fontsize'] = 13.
plt.rcParams['figure.figsize'] = [15.,8.]

In [ ]:
data = pd.read_csv('../input/water-potability/water_potability.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data.shape

## Missing Value with Missing Percentage:

### 1. Method using Function :

In [ ]:
def missing_value(data):
    value = data.isnull().sum().sort_values(ascending=False)
    percentage = (data.isnull().sum()*100/data.isnull().count()).sort_values(ascending=False)
    total_missing = pd.concat([value,percentage], axis=1, keys=['value', 'percentage'])
    return total_missing
missing_value(data)

### 2. Using List Comprehension

In [ ]:
# Or

nan_feature = [feature for feature in data.columns if data[feature].isna().sum()>=1]
for feature in nan_feature:
    print(f'{feature}, Value Missing {data[feature].isna().sum()}, Percentage Missing {data[feature].isna().sum()*100/data[feature].isna().count()}')

## Handing Missing Values:

In [ ]:
for feature in nan_feature:
    median_value = data[feature].median()
    data[feature].fillna(median_value, inplace=True)
data[nan_feature].isna().sum()

### Nan values got replaced by median value

In [ ]:
data[nan_feature].head()

In [ ]:
data.head()

## Unvariate, Bivariate and Multivariate Exploratory Data Analysis

### Dependent feature

In [ ]:
sns.countplot(data=data, x='Potability', palette='magma');

### From above, we can see the data Not portable qauality of water is more than portable
### It is a balanced dataset, no need to apply any methods to make it balance

## Dependent features

In [ ]:
data.columns

In [ ]:
m=2
for feature in data.columns[:9]:
    sns.histplot(data=data, x=feature,color=sns.color_palette()[m], kde=True)
    plt.grid()
    plt.show()
    sns.boxplot(data=data, x=feature, color=sns.color_palette()[m])
    plt.show()
    m=+1
    

### All features are having Outliers

## Handling Outliers

In [ ]:
for i in data.columns[:9]:
    IQR = data[i].quantile(0.75)-data[i].quantile(0.25)
    data.loc[data[i]>=data[i].quantile(0.75)+(IQR*1.5), i] = data[i].quantile(0.75)+(IQR*1.5)
    data.loc[data[i]<=data[i].quantile(0.25)-(IQR*1.5), i] = data[i].quantile(0.25)-(IQR*1.5)

In [ ]:
for feature in data.columns[:9]:
    sns.boxplot(data=data, x=feature, palette='Accent')
    plt.show()

## Outliers removed

In [ ]:
data.head()

## Corelation Matrix

In [ ]:
sns.heatmap(data.corr(), cmap='Accent', annot=True);

## Feature Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

feature_scale  = [feature for feature in data.columns if feature not in ['Potability']]
scaler = MinMaxScaler()
data[feature_scale] = scaler.fit_transform(data[feature_scale])

In [ ]:
data.head()

## Train Test Split

In [ ]:
x= data.drop(['Potability'], axis=1)
y = data['Potability']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2, random_state=0)

In [ ]:
len(x_train), len(y_train)

## Import Classification Models

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
models = {"Logistic Regression":LogisticRegression(),
          "RandomForestClassifier": RandomForestClassifier(),
          "Gradient Boosting": GradientBoostingClassifier(),
          "Naive Bayes": GaussianNB(),
          "AdaBoostClassifier": AdaBoostClassifier(),
          "KNN": KNeighborsClassifier()
         }
def fit_and_score(models, x_train, x_test, y_train, y_test):
    np.random.seed(42)
    
    model_scores = {}
    for name, model in models.items():
        model.fit(x_train,y_train)
        model_scores[name] = model.score(x_test, y_test)
        
    return model_scores

In [ ]:
score = fit_and_score(models= models,
                     x_train=x_train,
                     x_test=x_test,
                     y_train=y_train,
                     y_test=y_test)
score